In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm
from sklearn.metrics import silhouette_score

In [67]:
dataset = pd.read_csv('image_test_data.csv',delimiter=',',header = None)

In [68]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
dataset = dataset.loc[:, (dataset != 0).any(axis=0)]

In [70]:
dataset

,33,34,35,36,37,38,39,40,41,42,...,768,769,770,771,772,773,774,775,776,777
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
X = dataset.loc[:,:].values

In [72]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [73]:
class Kmeans:


    def __init__(self, n_cluster, max_iter=200, random_state=123):
        self.n_cluster = n_cluster
        self.max_iter = max_iter
        self.random_state = random_state

    def initial_centroids(self, X):
        np.random.RandomState(self.random_state)
        random_indx = np.random.permutation(X.shape[0])
        centroids = X[random_indx[:self.n_cluster]]
        return centroids
    
    def compute_dist(self, X, centroids):
        dist = np.zeros((X.shape[0], self.n_cluster))
        for k in range(self.n_cluster):
            row_norm = norm(X - centroids[k, :], axis=1)
            dist[:, k] = np.square(row_norm)
        return dist
    
    def compute_sse(self, X, labels, centroids):
        dist = np.zeros(X.shape[0])
        for k in range(self.n_cluster):
            dist[labels == k] = norm(X[labels == k] - centroids[k], axis=1)
        return np.sum(np.square(dist))
    

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_cluster, X.shape[1]))
        for k in range(self.n_cluster):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids


    def find_nearest_cluster(self, dist):
        return np.argmin(dist, axis=1)


    def fit(self, X):
        self.centroids = self.initial_centroids(X)
        for i in range(self.max_iter):
            old_centroids = self.centroids
            dist = self.compute_dist(X, old_centroids)
            self.labels = self.find_nearest_cluster(dist)
            self.centroids = self.compute_centroids(X, self.labels)
            if np.all(old_centroids == self.centroids):
                break
        self.error = self.compute_sse(X, self.labels, self.centroids)
    
    def predict(self, X):
        dist = self.compute_dist(X, old_centroids)
        return self.find_nearest_cluster(dist)

In [74]:
km = Kmeans(n_cluster=10,max_iter=200)

In [75]:
km.fit(X)

In [76]:
centroids = km.centroids

In [77]:
labels = km.labels

In [78]:
labels

array([5, 6, 3, ..., 5, 5, 8], dtype=int64)

In [79]:
labels = labels + 1

In [80]:
labels

array([6, 7, 4, ..., 6, 6, 9], dtype=int64)

In [81]:
np.savetxt('cluster_main1.dat', labels , fmt='%s')

In [85]:
score= silhouette_score(X, labels)

In [86]:
score

0.059163721859430085